# flowchart


<img src = 'crawler_flowchart.png'>

In [299]:
# import mysql.connector
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.common.keys import Keys
#from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import numpy as np
import sys
import re

### 1. crawrler 

In [10]:
driver = webdriver.Chrome('chromedriver.exe')

driver.get("http://www.google.com")

time.sleep(2)


element = driver.find_element_by_name('q')

element.clear()
element.send_keys('google map')
element.submit()

element = driver.find_element_by_xpath('//h3[@class="LC20lb DKV0Md"]')
element.click()

In [132]:
def get_info(html, shop_name, shop_address):
# shop 주소, 총 별점 갖고오기
    search_name, address, total_score, match=get_totalScore(html,shop_name, shop_address)
    
    # shop info가 있는가(addrsss로 확인)
    if address == '':
        review_list = ''
        score_list = ''
        return search_name, address,total_score, review_list, score_list, match 

    # 리뷰가 없는 경우    
    if  total_score == 0:
        review_list = 0
        score_list= 0
        
    # 리뷰가 있는 경우
    else:
        # 리뷰 text(버튼 )클릭
        review_button = driver.find_element_by_xpath('//button[@class="jqnFjrOWMVU__button gm2-caption"]')
        review_button.click()

        time.sleep(SCROLL_PAUSE_TIME)

        # 스크롤 다운
        try:
            element = driver.find_element_by_xpath('//span[@class="section-review-text"]')
            element.click()

        except Exception as err:
            print(err)

        i = 0
        while  True:
            i += 1
            html = driver.find_element_by_tag_name('html')
            time.sleep(SCROLL_PAUSE_TIME)

            text1 = html.text
            html.send_keys(Keys.END)
            time.sleep(SCROLL_PAUSE_TIME)

            html = driver.find_element_by_tag_name('html')
            text2 = html.text

            if text1 == text2:
                break
            else:
                continue
    
        review_html = driver.page_source

        review_list, score_list=get_review(review_html) 
        driver.back()

    return search_name, address, total_score, review_list, score_list, match

In [133]:
def get_review(html):
    soup = BeautifulSoup(html, 'html.parser')
    # 개별 리뷰
    total_review =soup.find_all('span',attrs = {'class': "section-review-text"} )

    review_list=[]
    for i, review in enumerate(total_review):
        review_list.append(review.text)

    #### 개별 별점
    scores =soup.find_all('span',attrs = {'class': "section-review-stars", 'role': 'img'} )

    score_list = []
    for score in scores:
        score_list.append(score.attrs['aria-label'])

    return review_list, score_list

In [403]:
def start_crwarling(k, forpet_hash,shop_name, shop_address):
    print(k, shop_name, shop_address)

    # shop 이름 입력
    element = driver.find_element_by_xpath('//input[@autofocus="autofocus"]')
    element.clear()
    
    search_word = shop_name + ' ' +shop_address[0]
#     search_word = shop_name
    element.send_keys(search_word)
    element.submit()
    
    search_button = driver.find_element_by_xpath('//button[@class="searchbox-searchbutton"]')
    search_button.click()
    
    time.sleep(SCROLL_PAUSE_TIME)

    # 키워드 입력 후, dict로 인덱스와 match 여부 받아오기
    select_shop_html = driver.page_source
    shop_select_dict= find_shop_by_name(select_shop_html, shop_name)
    
    # shoh이 여러개인 경우
    if len(shop_select_dict) != 0:
        print(f'shop이 {len(shop_select_dict)}개입니다')
        for index, name in shop_select_dict.items():
            shop_index =  2 * int(index) -1
            
            search_name = name
#             
            time.sleep(3)

            try:
                select_shop = driver.find_element_by_xpath(f'//div[@class = "section-layout section-scrollbox scrollable-y scrollable-show section-layout-flex-vertical"]/div/div[{shop_index}]/div[1]/div[1]')

                select_shop.click()

                time.sleep(SCROLL_PAUSE_TIME)

                html = driver.page_source

                search_name, address, total_score, review_list, score_list, match =get_info(html, shop_name, shop_address)
                print(f'{index}번째 shop:', search_name, address, total_score, match )
            except:
                search_name = shop_name
                address = ''
                total_score = ''
                match = 'not info'
                address = ''
                review_list = ''
                score_list = ''
           
            save_dataframe(forpet_hash, search_name, shop_address, address, total_score, review_list, score_list, match)
        
            time.sleep(SCROLL_PAUSE_TIME)
            driver.back()  
     
    # shop이 1개 인경우
    else:
        print('shop이 1개입니다')
        html = driver.page_source
        search_name, address, total_score, review_list, score_list, match =get_info(html, shop_name, shop_address)
        print(f'shop 1개, {search_name}, {address}, {match} ')
        
        save_dataframe(forpet_hash, search_name, shop_address, address, total_score, review_list, score_list, match)
    driver.back()  

    

In [ ]:
# 여러개의  shop 목록에서 주소로 찾기
def find_shop_by_name(select_shop_html, shop_name):
    soup = BeautifulSoup(select_shop_html, 'html.parser')
#     print(select_shop_html)
    all_shop_name = soup.find_all('div', attrs = {'class': "section-result-title-container"} )
    
    shop_select_dict = {}
    for i, shop in enumerate(all_shop_name):
        include_word = ['동물', '펫', '애견', 'dot','cat','도그','캣','pet','야옹','멍멍']
        name_include_word = sum([ word in shop.text  for word in include_word]) 
        
        # 이름이 같은 경우
        if shop_name == shop.text :
            shop_select_dict[str(i+1)] = [shop.text, 'same_name']  
        # 이름이 비슷한 경우
        elif name_include_word >= 1:
            shop_select_dict[str(i+1)] = [shop.text,'simiral_name']
        else:
            pass
#     print(shop_select_dict)
    return shop_select_dict

In [405]:
# 주소와 total 별점 가져오기
def get_totalScore(html,shop_name, shop_address):
    
    
    soup = BeautifulSoup(html, 'html.parser')

    time.sleep(SCROLL_PAUSE_TIME)
    infos =soup.find_all('span',attrs = {'class': "section-info-text"} )
    time.sleep(SCROLL_PAUSE_TIME)
    
    # 검색 결과가 있는 경우
    try: 
        search_name = soup.find_all('div', attrs = {'class':'section-hero-header-title-description' })[0].text.replace(' ','')
        if shop_name in search_name:
            search_name = shop_name
        else:
            p = re.compile("[가-힣]")
            search_name = ''.join(p.findall(search_name)).replace('주','')
        
        print('serach_name', search_name, len(search_name))
        print('shop_name', shop_name, len(shop_name))
        
        #  shop이름이 일치하는 경우
        if shop_name in search_name:
            # shop 정보가 없는 경우
            if len(infos)== 0:
                total_score = ''
                match = 'not info'
                address = ''

                return address, total_score
            # shop 정보가 있는 경우    
            else:
                address =infos[0].text

                gu = shop_address[0] in address
                dong = shop_address[1] in address
                bungi = shop_address[2] in address


                if (gu and dong and bungi) or (dong and bungi):
                    match = 'match_all'
                elif (gu and dong) or dong:
                    match = 'match_dong'
                elif gu:
                    match = 'match_gu'
                else:
                    match = 'not_match' 
                    
                    
                    
        #shop이름이 다른 경우
        else:
            address =infos[0].text
            match = 'x'


        total_score = soup.find_all('div',attrs = {'class': "gm2-display-2"} )

        if len(total_score)!=0:
            total_score = total_score[0].text
        else:
            total_score = 0
    
    # 검색 결과가 없는 경우
    except:
        total_score = ''
        match = 'not info'
        address = ''
        search_name = shop_name
    
    return search_name, address, total_score, match

In [406]:
def save_dataframe(forpet_hash, search_name, shop_address, address, total_score, review_list, score_list, match):
    
    # shop 정보가 없는 경우
    if address == '':
        not_info_table_insert(not_info_table, forpet_hash, search_name, match)
    
    # shop 정보가 있는 경우
    else:
        # match 여부에 따라
        if match == 'match_all':
            table_total = match_total_table
            table_review = match_review_table

        elif match == 'x':
            table_total = x_total_table
            table_review = x_review_table

        # shop 이름만 같은 경우, match = match_dong, match_gu, not_match
        else:    
            table_total = not_match_total_table
            table_review = not_match_review_table

        total_table_insert(table_total, forpet_hash, search_name, shop_address, address, total_score, review_list, match)
        review_table_insert(table_review, forpet_hash, search_name, shop_address, address, review_list, score_list, match)   


### 2. table
#### 1. total table

In [421]:
not_match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'address','total_score', 'review_num','match'])
not_match_total_table

,forpet_hash,shop_name,shop_address,address,total_score,review_num,match


In [422]:
match_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'address','total_score', 'review_num','match'])
match_total_table

,forpet_hash,shop_name,shop_address,address,total_score,review_num,match


In [423]:
x_total_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'shop_address', 'address','total_score', 'review_num','match'])
x_total_table

,forpet_hash,shop_name,shop_address,address,total_score,review_num,match


In [424]:
def total_table_insert(total_table, forpet_hash, search_name, shop_address, address, total_score, review_list, match):
    if len(total_table[total_table['address']== address]) == 0:
        # shop 정보가 없고, total_sclre가 없는 경우
        if total_score == '':
            total_table.loc[len(total_table)+1] = [forpet_hash, search_name, shop_address,  address, np.nan, np.nan, match]
        # shop 정보가 있고, total_score가 있는 경우
        elif total_score == 0:
            total_table.loc[len(total_table)+1] = [forpet_hash, search_name, shop_address, address, 0, 0, match]

        #total score이 없는 경우
        else:
            total_table.loc[len(total_table)+1] = [forpet_hash, search_name, shop_address, address,total_score, len(review_list), match]
        print(f'total_table: {search_name}, {address} 등록완료')
    else:
        print(f'total_table: {search_name}, {address} 이미 있음')
    return total_table

#### 2. review table

In [425]:
not_match_review_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name','shop_address','address', 'num', 'review', 'score', 'match'])
not_match_review_table

,forpet_hash,shop_name,shop_address,address,num,review,score,match


In [426]:
match_review_table= pd.DataFrame(columns = ['forpet_hash','shop_name','shop_address','address', 'num', 'review', 'score', 'match'])
match_review_table

,forpet_hash,shop_name,shop_address,address,num,review,score,match


In [427]:
x_review_table= pd.DataFrame(columns = ['forpet_hash', 'shop_name','shop_address','address', 'num', 'review', 'score', 'match'])
x_review_table

,forpet_hash,shop_name,shop_address,address,num,review,score,match


In [428]:
def review_table_insert(review_table, forpet_hash, search_name, shop_address, address, review_list, score_list, match):
    if len(review_table[review_table['address']== address]) == 0:
        # shop 정보가 없고, total_sclre가 없는 경우
        if review_list == '':
            review_table.loc[len(review_table)+1] = [forpet_hash, search_name, shop_address, address, 1,  np.nan, np.nan, match]
            print(f'review_table: {search_name}, {address} 등록완료')

        # shop 정보가 있고, total_score가 있는 경우
        elif review_list == 0:
            review_table.loc[len(review_table)+1] = [forpet_hash, search_name, shop_address, address, 1, 0, 0, match]
            print(f'review_table: {search_name}, {address} 등록완료')

        #total score이 없는 경우
        else:
            for i, (review, score, match) in enumerate(zip(review_list, score_list, match)):
                review_table.loc[i+1+len(review_table)] = [forpet_hash, search_name, shop_address, address, i, review, score, match]
            print(f'review_table: {search_name}, {address}, {len(review_list)} 등록완료')
    else:
        print(f'review_table: {search_name}, {address} 이미 있음')

    return review_table

#### 3.not_info_table

In [429]:
not_info_table = pd.DataFrame(columns = ['forpet_hash', 'shop_name', 'match'])
not_info_table

,forpet_hash,shop_name,match


In [430]:
def not_info_table_insert(not_info_table, forpet_hash, shop_name, match):
    not_info_table.loc[1+len(not_info_table)] = [forpet_hash, shop_name, match]
    print(f'not_info_table: {shop_name} 등록완료')
    return not_info_table

### 3. ordinary table

In [431]:
ordinary_table = pd.read_csv('mysql_forpet_shop.csv', index_col = 0)
ordinary_table.head()

,forpet_hash,address_name,place_name
0,4b793eb834462b029e435eaeb49433efc61755c9e15a24...,서울 종로구 평창동 530-8,쿠나이앤티
1,f5169c15a126e4059f0a89bb80fccd236693d5056d24ae...,서울 종로구 창신동 468-1,애견용품할인매장
2,aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb...,서울 종로구 창신동 436-64,펫클럽 청계점
3,2806f173e3a783ed14d74c9a83da2f62c34f45d470e893...,서울 종로구 홍지동 103-24,나니스펫푸드
4,aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c...,서울 종로구 필운동 137-5,서촌양품


In [432]:
len(ordinary_table)

8582

In [433]:
SCROLL_PAUSE_TIME = 2
all_cnt = 0
for k in range( len(ordinary_table)):
    forpet_hash, address_name, place_name= ordinary_table.loc[k] 
  
    shop_address =address_name.split(' ')[1:]
    shop_name = place_name.replace(' ','')
    start_crwarling(k, forpet_hash, shop_name, shop_address)
    
    print('\n')
    try:
        back_button = driver.find_element_by_xpath('//button[@class="ozj7Vb3wnYq__action-button-clickable"]')
        back_button.click()
    except:
        pass

0 쿠나이앤티 ['종로구', '평창동', '530-8']
shop이 1개입니다
serach_name 쿠나이앤티 5
shop_name 쿠나이앤티 5
shop 1개, 쿠나이앤티, 서울특별시 종로구 평창동, match_dong 
total_table: 쿠나이앤티, 서울특별시 종로구 평창동 등록완료
review_table: 쿠나이앤티, 서울특별시 종로구 평창동 등록완료


1 애견용품할인매장 ['종로구', '창신동', '468-1']
shop이 3개입니다
serach_name 애견용품할인매장 8
shop_name 애견용품할인매장 8
1번째 shop: 애견용품할인매장 창신동 315-1번지 1층 종로구 서울특별시 KR 0 match_dong
total_table: 애견용품할인매장, 창신동 315-1번지 1층 종로구 서울특별시 KR 등록완료
review_table: 애견용품할인매장, 창신동 315-1번지 1층 종로구 서울특별시 KR 등록완료
serach_name 애견용품할인마트 8
shop_name 애견용품할인매장 8
2번째 shop: 애견용품할인마트 서울특별시 종로구 창신동 436-64번지 0 x
total_table: 애견용품할인마트, 서울특별시 종로구 창신동 436-64번지 등록완료
review_table: 애견용품할인마트, 서울특별시 종로구 창신동 436-64번지 등록완료
serach_name 애견용품할인매장 8
shop_name 애견용품할인매장 8
3번째 shop: 애견용품할인매장 금호동1가 513번지 성동구 서울특별시 KR 0 not_match
total_table: 애견용품할인매장, 금호동1가 513번지 성동구 서울특별시 KR 등록완료
review_table: 애견용품할인매장, 금호동1가 513번지 성동구 서울특별시 KR 등록완료


2 펫클럽청계점 ['종로구', '창신동', '436-64']
shop이 1개입니다
serach_name 펫클럽청계점 6
shop_name 펫클럽청계점 6
shop 1개, 펫클럽청계점, 서울특별시 종로구 창신동 436-64번지 일신

serach_name 펫죤애완동물용품점 9
shop_name 애견용품도매상가 8
8번째 shop: 펫죤애완동물용품점 서울특별시 강서구 화곡4동 842-3 3.0 x
total_table: 펫죤애완동물용품점, 서울특별시 강서구 화곡4동 842-3 등록완료
review_table: 펫죤애완동물용품점, 서울특별시 강서구 화곡4동 842-3, 6 등록완료


32 마루펫 ['중구', '쌍림동', '151-11']
shop이 1개입니다
shop 1개, 마루펫, , not info 
not_info_table: 마루펫 등록완료


33 미니강아지 ['중구', '신당동', '373-20']
shop이 1개입니다
shop 1개, 미니강아지, , not info 
not_info_table: 미니강아지 등록완료


34 H펫현대시티아울렛동대문점 ['중구', '을지로6가', '17-2']
shop이 1개입니다
shop 1개, H펫현대시티아울렛동대문점, , not info 
not_info_table: H펫현대시티아울렛동대문점 등록완료


35 분독 ['중구', '광희동2가', '303-16']
shop이 1개입니다
serach_name 분독 2
shop_name 분독 2
shop 1개, 분독, 광희동 2 KR 서울특별시 중구 광희동2가 303-16번지 4층 주세창화학 호 가, match_all 
total_table: 분독, 광희동 2 KR 서울특별시 중구 광희동2가 303-16번지 4층 주세창화학 호 가 등록완료
review_table: 분독, 광희동 2 KR 서울특별시 중구 광희동2가 303-16번지 4층 주세창화학 호 가 등록완료


36 라찌 ['중구', '신당동', '429-14']
shop이 1개입니다
serach_name 라찌 2
shop_name 라찌 2
shop 1개, 라찌, 서울특별시 중구 신당동 429-14 번지 전층, match_all 
total_table: 라찌, 서울특별시 중구 신당동 429-14 번지 전층 등록완료
review_table: 라찌, 서

shop이 1개입니다
shop 1개, 애니밀, , not info 
not_info_table: 애니밀 등록완료


76 투스투스 ['용산구', '한강로1가', '243-6']
shop이 1개입니다
shop 1개, 투스투스, , not info 
not_info_table: 투스투스 등록완료


77 더세컨드로워&포코앤루시 ['용산구', '한남동', '758-33']
shop이 1개입니다
shop 1개, 더세컨드로워&포코앤루시, , not info 
not_info_table: 더세컨드로워&포코앤루시 등록완료


78 프랑소와펫동부이촌점 ['용산구', '이촌동', '300-26']
shop이 1개입니다
serach_name 프랑소와펫 5
shop_name 프랑소와펫동부이촌점 10
1번째 shop: 프랑소와펫 번지 상가 21 동 KR 서울특별시 용산구 이촌동 300-23 한강맨숀아파트 104호 0 x
total_table: 프랑소와펫, 번지 상가 21 동 KR 서울특별시 용산구 이촌동 300-23 한강맨숀아파트 104호 등록완료
review_table: 프랑소와펫, 번지 상가 21 동 KR 서울특별시 용산구 이촌동 300-23 한강맨숀아파트 104호 등록완료


79 솔티핫티 ['용산구', '한남동', '743-24']
shop이 1개입니다
serach_name 솔티핫티 4
shop_name 솔티핫티 4
shop 1개, 솔티핫티, 한남동 743-24번지 용산구 서울특별시 KR, match_all 
total_table: 솔티핫티, 한남동 743-24번지 용산구 서울특별시 KR 등록완료
review_table: 솔티핫티, 한남동 743-24번지 용산구 서울특별시 KR 등록완료


80 베럴즈 ['강남구', '신사동', '520']
shop이 1개입니다
serach_name 배럴즈의류점 6
shop_name 베럴즈 3
shop 1개, 배럴즈의류점, 서울특별시 강남구 신사동 551-16, x 
total_table: 배럴즈의류점, 서울특별시 강남구 신사동 551-16

not_info_table: 애완용품 등록완료


109 스프링펫중곡점 ['광진구', '중곡동', '193-42']
shop이 1개입니다
serach_name 스프링펫 4
shop_name 스프링펫중곡점 7
1번째 shop: 스프링펫 서울특별시 동작구 상도동 324-68번지 1층 0 x
total_table: 스프링펫, 서울특별시 동작구 상도동 324-68번지 1층 등록완료
review_table: 스프링펫, 서울특별시 동작구 상도동 324-68번지 1층 등록완료


110 슈슈독 ['광진구', '군자동', '27-2']
shop이 1개입니다
serach_name 슈슈독 3
shop_name 슈슈독 3
shop 1개, 슈슈독, 인천광역시 부평구 갈산동 369, not_match 
total_table: 슈슈독, 인천광역시 부평구 갈산동 369 등록완료
review_table: 슈슈독, 인천광역시 부평구 갈산동 369 등록완료


111 브라더펫구의점 ['광진구', '구의동', '202-5']
shop이 1개입니다
serach_name 브라더펫 4
shop_name 브라더펫구의점 7
shop 1개, 브라더펫, 구의동 202-5번지 1층 임애빌딩 광진구 서울특별시 KR, x 
total_table: 브라더펫, 구의동 202-5번지 1층 임애빌딩 광진구 서울특별시 KR 등록완료
review_table: 브라더펫, 구의동 202-5번지 1층 임애빌딩 광진구 서울특별시 KR 등록완료


112 봉쥬르고양이 ['광진구', '중곡동', '47-13']
shop이 1개입니다
shop 1개, 봉쥬르고양이, , not info 
not_info_table: 봉쥬르고양이 등록완료


113 몰리스펫샵자양점 ['광진구', '자양동', '227-7']
shop이 1개입니다
serach_name 몰리스펫샵애완동물전문점 12
shop_name 몰리스펫샵자양점 8
Message: element not interactable
  (Session info: chrome=80.0.3987.14

serach_name 강동시동물의료센터동물병원 13
shop_name 천호동강아지 6
5번째 shop: 강동시동물의료센터동물병원 서울특별시 강동구 성내동 천호대로 1122 3.9 x
total_table: 강동시동물의료센터동물병원, 서울특별시 강동구 성내동 천호대로 1122 등록완료
review_table: 강동시동물의료센터동물병원, 서울특별시 강동구 성내동 천호대로 1122, 9 등록완료
serach_name 경희프라자약국약국 9
shop_name 천호동강아지 6
6번째 shop: 경희프라자약국약국 서울특별시 강동구 천호2동 397-3 0 x
total_table: 경희프라자약국약국, 서울특별시 강동구 천호2동 397-3 등록완료
review_table: 경희프라자약국약국, 서울특별시 강동구 천호2동 397-3 등록완료
not_info_table: 천호동강아지 등록완료
serach_name 쥬라기동물병원동물병원 11
shop_name 천호동강아지 6
14번째 shop: 쥬라기동물병원동물병원 서울특별시 강동구 천호동 333-38 4.2 x
total_table: 쥬라기동물병원동물병원, 서울특별시 강동구 천호동 333-38 등록완료
review_table: 쥬라기동물병원동물병원, 서울특별시 강동구 천호동 333-38, 22 등록완료
serach_name 안단테동물병원동물병원 11
shop_name 천호동강아지 6
15번째 shop: 안단테동물병원동물병원 서울특별시 강동구 천호동 30-5 4.7 x
total_table: 안단테동물병원동물병원, 서울특별시 강동구 천호동 30-5 등록완료
review_table: 안단테동물병원동물병원, 서울특별시 강동구 천호동 30-5, 7 등록완료
serach_name 광나루한강공원공원 9
shop_name 천호동강아지 6
not_info_table: 천호동강아지 등록완료
not_info_table: 천호동강아지 등록완료
serach_name 강동구립천호도서관도서관 12
shop_name 천호동강아지 6
not_info_table

KeyboardInterrupt: 

In [54]:



print('match_total_table\n',match_total_table)
print('not_match_total_table\n',not_match_total_table)
print('match_review_table\n',match_review_table)
print('not_match_review_table\n',not_match_review_table)
print('x_total_table\n', x_total_table)
print('x_review_table\n', x_review_table)


match_total_table
                                          forpet_hash shop_name  \
1  aa26d23f20a9b5642a6689f2a2761ca470703ebbd514eb...   펫클럽 청계점   
2  2806f173e3a783ed14d74c9a83da2f62c34f45d470e893...    나니스펫푸드   
3  aa18fd6e2328f00ac2567b7a37a4db308ceee03a01924c...      서촌양품   
4  fb83c79c4e90759c7c48970c543e2f8671e1e39d3eb76d...      뚜리까까   

                    shop_address total_score  match  
1  서울특별시 종로구 창신동 436-64번지 일신파워빌딩         NaN  match  
2   홍지동 103-24번지 1층 종로구 서울특별시 KR         NaN  match  
3            서울특별시 종로구 필운동 137-5         NaN  match  
4     서울특별시 성북구 동소문동3가 동소문로2길 79         NaN  match  
not_match_total_table
                                          forpet_hash    shop_name  \
1  f5169c15a126e4059f0a89bb80fccd236693d5056d24ae...     애견용품할인매장   
2  8aa63cf25e3842284a45ab5d4f603e816dba32c9406b88...      푸드마스터그룹   
3  bd8ce4fe5ec9031f0d986ac8e3091594a104e02d9e1d71...         도그스푼   
4  ad85ca27faea5e1533bdd8873a5a366d237ffd1311f510...    스프링펫 서대문점   
5  57eed23d4